In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/06/17 18:44:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/17 18:44:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [6]:
client.list_artifacts('729206dff5974b2aa8e138af3125d40a')

[<FileInfo: file_size=None, is_dir=True, path='artifacts'>]

In [7]:
client.create_experiment("experiment_thru_cli")

'6'

In [8]:
from mlflow.entities import ViewType
runs = client.search_runs(experiment_ids='5',
                          filter_string="",
                          run_view_type=ViewType.ACTIVE_ONLY,
                          max_results=5,
                          order_by=["metrics.rmse ASC"])

In [10]:
for run in runs:
    print(f"Run ID: {run.info.run_id}, RMSE: {run.data.metrics['rmse']:.4f}")

Run ID: 729206dff5974b2aa8e138af3125d40a, RMSE: 6.3422
Run ID: 5c29a0017716468392c99b9a8a202d19, RMSE: 6.3422
Run ID: ec94614cb6e041fdb71fc080cb2ec135, RMSE: 6.3422
Run ID: eb841f61768b480ca0397d8612277285, RMSE: 6.3422
Run ID: 070dfa0ac33d47668862bd491eb2415d, RMSE: 6.3422


In [23]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [24]:
run_id = "729206dff5974b2aa8e138af3125d40a"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2025/06/17 19:38:29 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/17 19:38:29 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'nyc-taxi-regressor'.
2025/06/17 19:38:29 WARNING mlflow.tracking._model_registry.fluent: Run with id 729206dff5974b2aa8e138af3125d40a has no artifacts at artifact path 'model', registering model based on models:/m-f5ed874a3f404629bb78a7aa86d80eec instead
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1750189109859, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1750189109859, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729206dff5974b2aa8e138af3125d40a', run_link=None, source='models:/m-f5ed874a3f404629bb78a7aa86d80eec', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [25]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_35937/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [27]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_35937/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1750189109859, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1750189180848, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729206dff5974b2aa8e138af3125d40a', run_link=None, source='models:/m-f5ed874a3f404629bb78a7aa86d80eec', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [28]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1750189109859, current_stage='Staging', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-06-17', last_updated_timestamp=1750189197716, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729206dff5974b2aa8e138af3125d40a', run_link=None, source='models:/m-f5ed874a3f404629bb78a7aa86d80eec', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [29]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import mlflow
import numpy as np

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": np.sqrt(mean_squared_error(y_test, y_pred))}

In [30]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [31]:
client.download_artifacts(run_id='729206dff5974b2aa8e138af3125d40a', path='02-experiment-tracking/mlruns/5/models/m-f5ed874a3f404629bb78a7aa86d80eec/artifacts/model.xgb', dst_path='.')

In [34]:
import pickle

with open("02-experiment-tracking/models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: '02-experiment-tracking/models/preprocessor.b'